## Hartree Fock en el átomo de Helio


In [5]:
import numpy as np



En este laboratorio vamos a implementar una rutina que permita obtener
la energía hartree-fock del átomo de helio a través de las ecuaciones
de Roothaan. Para eso vamos a expresar el orbital 1s del helio en terminos
de dos funciones bases:

\begin{equation}
\phi_1(r) = 2\alpha_1^{3/2}e^{-\alpha_1 r}Y^0_0, \qquad \phi_2(r) = 2\alpha_2^{3/2}*e^{-\alpha_2 r}Y^0_0,
\end{equation}

Los exponentes $ \alpha $ son:

\begin{equation}
\alpha_1 = 1.40 \qquad \alpha_2 = 2.90
\end{equation}

In [6]:
#alpha_1 = 1.45
#alpha_2 = 2.91

alpha_1 = 1.453
alpha_2 = 2.911

# Paso 0: Formar matrices $H^{core}$ y de Solapamiento

El primer paso para el calculo Hartree-Fock-Roothaan es calcular la matriz de solapamiento($S_{\mu\nu}$)
y del Hamiltoniano de core ($H^{core}_{\mu\nu}$):

Es fácil verificar que la integral de solpamiento esta dada por:

\begin{equation}
S_{12} = S_{21} = \frac{8\alpha_1^{3/2}\alpha_2^{3/2}}{(\alpha_1+\alpha_2)^3}
\end{equation}

y
\begin{equation}
S_{11} = S_{22} = 1
\end{equation}


In [7]:
S_12 = (8.0*alpha_1**(3./2.)*alpha_2**(3./2.))/(alpha_1+alpha_2)**(3.0)
S_21 = S_12
S_11 = 1
S_22 = S_11

S = np.array = ([[S_11,S_12],[S_21,S_22]])

Los elemento diagonales del Hamiltoniano de core estan dado por:

\begin{equation}
H_{ii} =  0.5\alpha_i^2 - 2\alpha_i 
\end{equation}

y los elementos fuera de la diagonal son:

\begin{equation}
H_{12} = H_{21} = \frac{\alpha_1^{3/2}\alpha_2^{3/2}(4\alpha_1\alpha_2 - 8\alpha_1-8\alpha_2)}{(\alpha_1 + \alpha_2)^3}
\end{equation}


In [8]:
H_11 = 0.5*alpha_1**2 - 2*alpha_1
H_22 = 0.5*alpha_2**2 - 2*alpha_2
H_12 = 4*alpha_1**(3./2.)*alpha_2**(3./2.)*(alpha_1*alpha_2 - 2.0*alpha_1 - 2.0*alpha_2)/(alpha_1+alpha_2)**3.
H_21 = H_12
H = np.array = ([[H_11,H_12],[H_21,H_22]])


Para poder calcular la matriz de Fock, necesitamos calcular las integrales de dos electrones:

# Paso 1: Calcular las integrales de repulsión electrónica (ERI)

Determine las integrales únicas $<\mu\lambda|\nu\sigma>$  de dos electrones para el átomo de He con dos funciones base, y guardelas en un tensor de cuatro indices ERI.



In [16]:
# Integrales de dos electrones:
eri1=5./8.*alpha_1 #<11|11>
eri2=5./8.*alpha_2 #<22|22>
eri3=(alpha_1**4*alpha_2+4*alpha_1**3*alpha_2**2+4*alpha_1**2*alpha_2**3+alpha_1*alpha_2**4)/(alpha_1+alpha_2)**4 #<12|12 >
eri4=20*alpha_1**3*alpha_2**3/(alpha_1+alpha_2)**5 #<11|22>
eri5=16*alpha_1**(9./2.)*alpha_2**(3./2.)/(3*alpha_1+alpha_2)**4*((12*alpha_1+8*alpha_2)/(alpha_1+alpha_2)**2+(9.*alpha_1+alpha_2)/(2.*alpha_1**2))#<11|12>
eri6=16*alpha_2**(9./2.)*alpha_1**(3./2.)/(3*alpha_2+alpha_1)**4*((12*alpha_2+8*alpha_1)/(alpha_2+alpha_1)**2+(9.*alpha_2+alpha_1)/(2.*alpha_2**2))#<22|21>
ERI=np.empty([2,2,2,2])
ERI[0][0][0][0] = eri1
ERI[0][0][0][1] = eri2
    # Construción del tensor de dos electrones



# Paso 2: Construir la matriz de transformción

Para poder emepzar con la iteración, debemos pasar a la base ortogonal de las funciones base y así tratar
la ecuación matricial de Hartree-Fock-Roothaan como un problema de valores propios. Procedemos a diagonalizar la matriz de solapamiento y obtener la matriz diagonal s.


\begin{equation}
\Large
\mathbf{s} = \mathbf{U^{\dagger}}\mathbf{S}\mathbf{U}
\end{equation}


Ahora que es una matriz diagonal, le podemos calcular el inverso de la raiz cuadrada: $\quad\mathbf{s^{-1/2}}$


Finalmente construimos la matriz de transformación de base $ \mathbf{S^{-1/2}} $ en 
la base original, utilizando la matriz de vecotres propios $\mathbf{U}$ de 
transformación de base.

\begin{equation}
\Large
\mathbf{S^{-1/2}} = \mathbf{U}\mathbf{s^{-1/2}}\mathbf{U^{\dagger}}
\end{equation}

# Paso 3: Construir la matriz de Fock inicial y encontrar la matriz de coeficientes en la base ortogonal

Ahora podemos formar la matriz de Fock, que en la primer iteración es la matriz del Hamiltoniano
core:



\begin{equation}
\Large
\mathbf{F} = \mathbf{H^{core}}
\end{equation}

Para poder solucionar el problema de valores propios debemos transformar la matriz de Fock
a la base de orbitales ortogonales:



\begin{equation}
\Large
\mathbf{F^{'}} = \mathbf{S^{\dagger -1/2}}\mathbf{F}\mathbf{S^{-1/2}}
\end{equation}


En esta base se cumple que:



\begin{equation}
\Large
\mathbf{F^{'}C^{'}} = \mathbf{C^{'}\epsilon}
\end{equation}




# Paso 4: Volver a la base original y formar la matriz de Densidad inicial:

Para obtener los coeficientes en la base original debemos transformar de vuelta, utilizando la matriz 
de transformación simétrica:

\begin{equation}
\Large
\mathbf{C} = \mathbf{S^{-1/2}}\mathbf{C^{'}}
\end{equation}


Con los coeficientes en la base original, es posible caluclar la matriz densidad. Nótese 
que la suma para construir la matriz densidad es sobre los electrones y no sobre las funciones base:


\begin{equation}
\Large
D_{\mu\nu} = 2\sum_{i=1}^{N/2} C_{\mu i}C_{\nu i} 
\end{equation}


# Paso 5: Calcular la energía HF inicial


Ahora podemos calcular la energía Hartree-Fock inicial:

\begin{equation}
\Large
E_{HF} = 0.5\sum_{\mu \nu}^{AO} D_{\mu \nu}(H_{\mu \nu} + F_{\mu \nu})
\end{equation}

Con los coeficientes nuevos podemos construir la matriz de Fock inicial, esta vez utliziando las integrales de repulsión electrónica: 

\begin{equation}
\Large
F_{\lambda \sigma} = \sum_{\mu \nu}^{AO} H_{\mu \nu} + D_{\mu \nu}(<\mu\sigma|\nu\lambda> - 0.5<\mu\sigma|\lambda\nu>)
\end{equation}


# Paso 5: Calcular la matriz de Fock nueva y repetri los pasos anterioes

Con esta matriz de Fock repetimos los pasos anteriores para obtener una nueva energía electrónica:

# Paso 6: Iterar hasta alcanzar la convergencia:
Finalmente, iteramos hasta alcanzar convergencia.

In [18]:
i = 1
E_thresh = 1e-8